In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 100
num_filter = 12
compression = 0.5
dropout_rate = 0.2
val_batch_size=60

In [9]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train[1])



170500096/170498071 [==============================] - 32s 0us/step
[[[154 177 187]
  [126 137 136]
  [105 104  95]
  ...
  [ 91  95  71]
  [ 87  90  71]
  [ 79  81  70]]

 [[140 160 169]
  [145 153 154]
  [125 125 118]
  ...
  [ 96  99  78]
  [ 77  80  62]
  [ 71  73  61]]

 [[140 155 164]
  [139 146 149]
  [115 115 112]
  ...
  [ 79  82  64]
  [ 68  70  55]
  [ 67  69  55]]

 ...

 [[175 167 166]
  [156 154 160]
  [154 160 170]
  ...
  [ 42  34  36]
  [ 61  53  57]
  [ 93  83  91]]

 [[165 154 128]
  [156 152 130]
  [159 161 142]
  ...
  [103  93  96]
  [123 114 120]
  [131 121 131]]

 [[163 148 120]
  [158 148 122]
  [163 156 133]
  ...
  [143 133 139]
  [143 134 142]
  [143 133 144]]]


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0
l = 30
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [20]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 12)   324         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 12)   48          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 12)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

activation_121 (Activation)     (None, 8, 8, 42)     0           batch_normalization_121[0][0]    
__________________________________________________________________________________________________
conv2d_122 (Conv2D)             (None, 8, 8, 6)      2268        activation_121[0][0]             
__________________________________________________________________________________________________
concatenate_115 (Concatenate)   (None, 8, 8, 48)     0           concatenate_114[0][0]            
                                                                 conv2d_122[0][0]                 
__________________________________________________________________________________________________
batch_normalization_122 (BatchN (None, 8, 8, 48)     192         concatenate_115[0][0]            
__________________________________________________________________________________________________
activation_122 (Activation)     (None, 8, 8, 48)     0           batch_normalization_122[0][0]    
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1),
              metrics=['accuracy'])


In [22]:
# model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))
train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=25,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=25, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  


Epoch 1/25
390/390 [==============================] - 527s 1s/step - loss: 1.7391 - acc: 0.3685 - val_loss: 1.5664 - val_acc: 0.4221
Epoch 2/25
284/390 [====================>.........] - ETA: 2:03 - loss: 1.4278 - acc: 0.4813390/390 [==============================] - 495s 1s/step - loss: 1.4019 - acc: 0.4909 - val_loss: 1.8041 - val_acc: 0.3952
Epoch 3/25
390/390 [==============================] - 494s 1s/step - loss: 1.2117 - acc: 0.5610 - val_loss: 1.3486 - val_acc: 0.5350
Epoch 4/25
 21/390 [>.............................] - ETA: 7:07 - loss: 1.1351 - acc: 0.5781390/390 [==============================] - 493s 1s/step - loss: 1.0686 - acc: 0.6171 - val_loss: 1.0377 - val_acc: 0.6320
Epoch 5/25
326/390 [========================>.....] - ETA: 1:14 - loss: 0.9811 - acc: 0.6469390/390 [==============================] - 493s 1s/step - loss: 0.9748 - acc: 0.6492 - val_loss: 1.2879 - val_acc: 0.5669
Epoch 6/25
390/390 [==============================] - 492s 1s/step - loss: 0.9024 - acc: 0.6

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=12,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)
# determine Loss function and Optimizer


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=12, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  if sys.path[0] == '':


Epoch 1/12
390/390 [==============================] - 510s 1s/step - loss: 0.3696 - acc: 0.8701 - val_loss: 0.5085 - val_acc: 0.8305
Epoch 2/12
284/390 [====================>.........] - ETA: 2:02 - loss: 0.3501 - acc: 0.8783390/390 [==============================] - 494s 1s/step - loss: 0.3501 - acc: 0.8775 - val_loss: 0.5089 - val_acc: 0.8315
Epoch 3/12
390/390 [==============================] - 494s 1s/step - loss: 0.3402 - acc: 0.8825 - val_loss: 0.5136 - val_acc: 0.8300
Epoch 4/12
 21/390 [>.............................] - ETA: 7:09 - loss: 0.3474 - acc: 0.8776390/390 [==============================] - 494s 1s/step - loss: 0.3375 - acc: 0.8825 - val_loss: 0.5070 - val_acc: 0.8347
Epoch 5/12
326/390 [========================>.....] - ETA: 1:14 - loss: 0.3301 - acc: 0.8858390/390 [==============================] - 494s 1s/step - loss: 0.3316 - acc: 0.8859 - val_loss: 0.5132 - val_acc: 0.8315
Epoch 6/12
390/390 [==============================] - 494s 1s/step - loss: 0.3237 - acc: 0.8

In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001),
              metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=len(x_train) , epochs=12,
                   validation_data=val_datagen.flow(x_test,y_test,batch_size=val_batch_size)
                   ,verbose=1)
# determine Loss function and Optimizer

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=12, validation_data=<keras.pre..., verbose=1, steps_per_epoch=390)`
  if sys.path[0] == '':


Epoch 1/12
390/390 [==============================] - 511s 1s/step - loss: 0.2966 - acc: 0.8968 - val_loss: 0.4997 - val_acc: 0.8395
Epoch 2/12
284/390 [====================>.........] - ETA: 2:02 - loss: 0.2951 - acc: 0.8960390/390 [==============================] - 494s 1s/step - loss: 0.3004 - acc: 0.8950 - val_loss: 0.5002 - val_acc: 0.8386
Epoch 3/12
390/390 [==============================] - 493s 1s/step - loss: 0.2979 - acc: 0.8962 - val_loss: 0.5023 - val_acc: 0.8383
Epoch 4/12
 21/390 [>.............................] - ETA: 7:08 - loss: 0.3097 - acc: 0.8947390/390 [==============================] - 492s 1s/step - loss: 0.2971 - acc: 0.8964 - val_loss: 0.4973 - val_acc: 0.8404
Epoch 5/12
326/390 [========================>.....] - ETA: 1:14 - loss: 0.2912 - acc: 0.8977390/390 [==============================] - 493s 1s/step - loss: 0.2923 - acc: 0.8977 - val_loss: 0.4974 - val_acc: 0.8393
Epoch 6/12
390/390 [==============================] - 493s 1s/step - loss: 0.2971 - acc: 0.8

In [26]:
model.evaluate(x_test/255.0,y_test,verbose=1)

10000/10000 [==============================] - 52s 5ms/step


[0.4965174989700317, 0.8409]

In [27]:
# Test the model
score = model.evaluate(x_test/255.0, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 52s 5ms/step
Test loss: 0.4965174989700317
Test accuracy: 0.8409


In [28]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_improved.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_improved.h5')